In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[3],3a
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.37556e-24
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}",co2
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 2, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.387582                    1  0.251587
0.000750        2  0.387546                    2  0.256587
0.001052        3  0.400928                    3  0.269264
0.001476        4  0.412757                    4  0.281879
0.002070        5  0.422496                    5  0.294185
0.002904        6  0.428964                    6  0.305671
0.004074        7  0.431441                    7  0.316105
0.005714        8  0.428594                    8  0.324703
0.008015        9  0.419080                    9  0.330530
0.011243       10  0.401382                   10  0.332535
0.015771       11  0.373889                   11  0.328838
0.022122       12  0.334802                   12  0.317106
0.031031       13  0.282050                   13  0.294062
0.043528       14  0.217212                   14  0.260269
0.061057       15  0.193478                   15  0.258687
0.085645       16  0.237149                   16  0.310882
0.120136       17  0.294388                   17  0.367102
0.168516       18  0.335502                   18  0.391524
0.236378       19  0.370358                   19  0.392014
0.331549       20  0.415121                   20  0.402987
0.465100       21  0.494583                   21  0.472663
0.652400       22  0.622851                   22  0.618243
0.915100       23  0.624268                   23  0.658386
1.283650       24  0.524220                   24  0.586342
1.800600       25  0.441412                   25  0.505027
2.525700       26  0.355842                   26  0.414910
3.542800       27  0.288916                   27  0.351002
4.969550       28  0.236564                   28  0.303774
6.970850       29  0.192368                   29  0.260656
9.778100       30  0.154051                   30  0.211640
13.715850      31  0.120743                   31  0.161741
19.239350      32  0.092200                   32  0.116377
26.987250      33  0.068633                   33  0.079985
37.855300      34  0.050726                   34  0.056054
53.100050      35  0.025755                   35  0.029744
73.887500      36 -0.003896                   36 -0.003713
97.662500      37 -0.022584                   37 -0.027252
121.437500     38 -0.019633                   38 -0.023354
145.212500     39 -0.003142                   39 -0.002700
168.987500     40  0.007541                   40  0.011680
192.762500     41  0.017182                   41  0.025636
216.537500     42  0.026291                   42  0.041438
240.312500     43  0.034933                   43  0.059988
264.087500     44  0.043120                   44  0.080988
287.862500     45  0.050840                   45  0.102715
311.637500     46  0.058081                   46  0.125325
335.412500     47  0.064840                   47  0.150133
359.187500     48  0.071120                   48  0.172865
382.962500     49  0.076922                   49  0.186648
406.737500     50  0.082269                   50  0.203070
430.512500     51  0.087213                   51  0.224737
454.287500     52  0.091786                   52  0.248806
478.062500     53  0.096022                   53  0.271773
501.837500     54  0.099969                   54  0.291545
525.612500     55  0.103684                   55  0.310888
549.387500     56  0.107224                   56  0.329413
573.162500     57  0.110673                   57  0.347597
596.937500     58  0.114144                   58  0.363750
620.712500     59  0.117747                   59  0.376223
644.487500     60  0.121624                   60  0.385632
668.262500     61  0.125609                   61  0.392076
692.037500     62  0.128635                   62  0.394040
715.812500     63  0.128751                   63  0.388652
739.587500     64  0.125153                   64  0.391360
763.362500     65  0.119559                   65  0.371915
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -30.870337   0.000000 -30.870337 -25.455290  4.438955e-08   
0.000624    2     -30.870355   0.000046 -30.870308 -25.455308  3.620268e-05   
0.000876    3     -30.870362   0.000065 -30.870297 -25.455315  5.089270e-05   
0.001229    4     -30.870372   0.000092 -30.870280 -25.455325  7.192247e-05   
0.001723    5     -30.870386   0.000130 -30.870256 -25.455338  1.020213e-04   
0.002417    6     -30.870404   0.000183 -30.870221 -25.455357  1.450475e-04   
0.003391    7     -30.870431   0.000259 -30.870172 -25.455383  2.064623e-04   
0.004757    8     -30.870467   0.000365 -30.870102 -25.455420  2.939561e-04   
0.006672    9     -30.870517   0.000512 -30.870005 -25.455470  4.182373e-04   
0.009359    10    -30.870586   0.000715 -30.869871 -25.455541  5.940686e-04   
0.013128    11    -30.870684   0.000992 -30.869692 -25.455640  8.415560e-04   
0.018415    12    -30.870822   0.001364 -30.869458 -25.455780  1.187392e-03   
0.025830    13    -30.871020   0.001856 -30.869164 -25.455979  1.665445e-03   
0.036232    14    -30.871310   0.002494 -30.868816 -25.456268  2.316458e-03   
0.050823    15    -30.871748   0.003307 -30.868441 -25.456692  3.190850e-03   
0.071291    16    -30.872394   0.004423 -30.867971 -25.457300  4.425838e-03   
0.100000    17    -30.873284   0.006119 -30.867165 -25.458112  6.294856e-03   
0.140271    18    -30.874444   0.008683 -30.865761 -25.459135  9.069079e-03   
0.196760    19    -30.875906   0.012390 -30.863515 -25.460367  1.292130e-02   
0.275997    20    -30.877654   0.017614 -30.860039 -25.461751  1.798585e-02   
0.387100    21    -30.879536   0.024959 -30.854577 -25.463094  2.463313e-02   
0.543100    22    -30.880927   0.035491 -30.845437 -25.463768  3.404236e-02   
0.761700    23    -30.879840   0.050533 -30.829308 -25.461885  4.816995e-02   
1.068500    24    -30.875457   0.068836 -30.806621 -25.456658  6.687255e-02   
1.498800    25    -30.868780   0.088880 -30.779900 -25.448908  8.901339e-02   
2.102400    26    -30.859879   0.111540 -30.748338 -25.438598  1.148168e-01   
2.949000    27    -30.849519   0.136866 -30.712653 -25.426332  1.441649e-01   
4.136600    28    -30.837909   0.165900 -30.672008 -25.412091  1.793072e-01   
5.802500    29    -30.824962   0.199637 -30.625325 -25.395634  2.228025e-01   
8.139200    30    -30.810726   0.238648 -30.572078 -25.376762  2.760874e-01   
11.417000   31    -30.795474   0.283211 -30.512264 -25.355808  3.373175e-01   
16.014700   32    -30.779839   0.333336 -30.446503 -25.333698  4.033050e-01   
22.464000   33    -30.764972   0.388906 -30.376065 -25.312497  4.710210e-01   
31.510500   34    -30.752427   0.449910 -30.302516 -25.295271  5.395184e-01   
44.200100   35    -30.742087   0.515821 -30.226266 -25.283661  6.121758e-01   
62.000000   36    -30.746706   0.574745 -30.171962 -25.296441  6.876784e-01   
85.775000   37    -30.798906   0.615973 -30.182933 -25.378518  7.592980e-01   
109.550000  38    -30.900056   0.653519 -30.246537 -25.525063  8.290847e-01   
133.325000  39    -31.038851   0.737021 -30.301830 -25.712708  9.509513e-01   
157.100000  40    -31.196946   0.886269 -30.310677 -25.913773  1.144411e+00   
180.875000  41    -31.371584   1.082145 -30.289439 -26.124536  1.388073e+00   
204.650000  42    -31.560529   1.319481 -30.241049 -26.343893  1.679635e+00   
228.425000  43    -31.761821   1.594818 -30.167004 -26.574309  2.026766e+00   
252.200000  44    -31.973608   1.904987 -30.068620 -26.815981  2.437401e+00   
275.975000  45    -32.194332   2.247150 -29.947182 -27.069707  2.919240e+00   
299.750000  46    -32.422620   2.618619 -29.804001 -27.336785  3.475625e+00   
323.525000  47    -32.657242   3.016817 -29.640426 -27.618594  4.110426e+00   
347.300000  48    -32.897094   3.439278 -29.457816 -27.916803  4.831500e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')